In [1]:
!pip install pulp
!pip install requests

This model was generated to do an initial test on initial players to choose from FPL historical data

This needs to be added to the main FPL note book ultimately so there is one source of information

Notebook script to import FPL data, and put into a ML model to optimise team selection

In [2]:
import requests
import pulp as lp

# Define the FPL API URL for player data
fpl_api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# Make a request to the FPL API to fetch player data
response = requests.get(fpl_api_url)
data = response.json()

# Extract player data from the API response
players_data = {}
teams_data = {}  # Store team names
for player in data["elements"]:
    player_name = f"{player['first_name']} {player['second_name']}"
    players_data[player_name] = {
        "position": player["element_type"],
        "points_total": player["total_points"],
        "cost": player["now_cost"] / 10,  # Convert cost to millions
        "team": player["team"],
    }
    team_id = player["team"]
    team_name = data["teams"][team_id - 1]["name"]  # Adjust team_id to index
    teams_data[team_id] = team_name

# Define your total budget in millions
total_budget = 100  # Updated maximum budget to £100 million

# Define the maximum and minimum number of players allowed for each position in the starting 11
max_starting_11_players_per_position = {
    1: 1,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward (Striker)
}

min_starting_11_players_per_position = {
    1: 1,  # Minimum 1 Goalkeeper for starting 11
    2: 3,  # Minimum 3 Defenders for starting 11
    3: 2,  # Minimum 2 Midfielders for starting 11
    4: 1,  # Minimum 1 Forward (Striker) for starting 11
}

# Define the maximum number of players allowed for each position in the squad
max_squad_players_per_position = {
    1: 2,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward (Striker)
}

# Create a PuLP linear programming problem
model = lp.LpProblem("FantasyFootballTeam", lp.LpMaximize)

# Create a binary variable for each player to represent if they are in the starting 11
player_vars_starting_11 = {player: lp.LpVariable(f"starting_11_{player}", cat=lp.LpBinary) for player in players_data.keys()}
# Create a binary variable for each player to represent if they are in the squad
player_vars_squad = {player: lp.LpVariable(f"squad_{player}", cat=lp.LpBinary) for player in players_data.keys()}

# Objective function: maximize total points from the starting 11
model += lp.lpSum(players_data[player]["points_total"] * player_vars_starting_11[player] for player in players_data.keys()), "TotalPointsStarting11"

# Constraints
# Budget constraint for the whole squad
model += lp.lpSum(players_data[player]["cost"] * player_vars_squad[player] for player in players_data.keys()) <= total_budget, "Budget"

# Squad size constraint
model += lp.lpSum(player_vars_squad[player] for player in players_data.keys()) == 15, "SquadSize"

# Ensure the starting 11 is part of the squad
for player in players_data.keys():
    model += player_vars_starting_11[player] <= player_vars_squad[player], f"Starting11InSquad_{player}"

# Ensure exactly 11 players in the starting 11
model += lp.lpSum(player_vars_starting_11[player] for player in players_data.keys()) == 11, "Starting11Size"

# Position constraints for the starting 11
for position, min_count in min_starting_11_players_per_position.items():
    model += lp.lpSum(player_vars_starting_11[player] for player in players_data.keys() if players_data[player]["position"] == position) >= min_count, f"MinStarting11PlayersPosition{position}"

for position, max_count in max_starting_11_players_per_position.items():
    model += lp.lpSum(player_vars_starting_11[player] for player in players_data.keys() if players_data[player]["position"] == position) <= max_count, f"MaxStarting11PlayersPosition{position}"

# Maximum players per position for the squad
for position, max_count in max_squad_players_per_position.items():
    model += lp.lpSum(player_vars_squad[player] for player in players_data.keys() if players_data[player]["position"] == position) <= max_count, f"MaxSquadPlayersPosition{position}"

# Constraint: no more than 3 players from any single team in the squad
for team_id, team_name in teams_data.items():
    model += lp.lpSum(player_vars_squad[player] for player in players_data.keys() if players_data[player]["team"] == team_id) <= 3, f"MaxPlayersPerTeam_{team_name}"

# Solve the ILP problem
model.solve()

# Create a dictionary to store selected players by position
selected_players_by_position = {
    1: [],  # Goalkeeper
    2: [],  # Defender
    3: [],  # Midfielder
    4: [],  # Forward (Striker)
}

# Populate the dictionary with selected players
for player in players_data.keys():
    if lp.value(player_vars_squad[player]) > 0:
        data = players_data[player]
        selected_players_by_position[data["position"]].append(player)

# Create a list of all selected players
selected_players = [player for player in players_data.keys() if lp.value(player_vars_squad[player]) > 0]

# Extract the starting 11 players
starting_11 = [player for player in players_data.keys() if lp.value(player_vars_starting_11[player]) > 0]

# Select the remaining players for the bench
bench_players = [player for player in selected_players if player not in starting_11]

# Sort starting 11 and bench players by position
starting_11_sorted = sorted(starting_11, key=lambda x: players_data[x]["position"])
bench_players_sorted = sorted(bench_players, key=lambda x: players_data[x]["position"])

# Print the selected players and their data sorted by position
for position in selected_players_by_position.keys():
    position_name = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward (Striker)",
    }.get(position, "Unknown")
    print(f"Position: {position_name}")
    for player in selected_players_by_position[position]:
        data = players_data[player]
        team_name = teams_data[data["team"]]  # Get team name from teams_data
        print(f"Player: {player} - Points: {data['points_total']} - Cost: £{data['cost']} million - Team: {team_name}")

# Suggest the captain based on the player with the highest points
captain = max(starting_11, key=lambda x: players_data[x]["points_total"])

print(f"\nSuggested Captain (Double Points): {captain}")

# Generate total points of the starting 11
total_points_starting_11 = sum(players_data[player]["points_total"] for player in starting_11)

# Output the results with positions
position_names = {
    1: "Goalkeeper",
    2: "Defender",
    3: "Midfielder",
    4: "Forward (Striker)",
}

print("\nStarting 11 Players:")
for player in starting_11_sorted:
    data = players_data[player]
    team_name = teams_data[data["team"]]
    position = position_names[data["position"]]
    print(f"Player: {player} - Position: {position} - Points: {data['points_total']} - Cost: £{data['cost']} million - Team: {team_name}")

print("\nBench Players:")
for player in bench_players_sorted:
    data = players_data[player]
    team_name = teams_data[data["team"]]
    position = position_names[data["position"]]
    print(f"Player: {player} - Position: {position} - Points: {data['points_total']} - Cost: £{data['cost']} million - Team: {team_name}")

print(f"\nTotal Points for Starting 11: {total_points_starting_11}")


Position: Goalkeeper
Player: Jordan Pickford - Points: 153 - Cost: £4.8 million - Team: Everton
Player: Caoimhin Kelleher - Points: 34 - Cost: £3.6 million - Team: Liverpool
Position: Defender
Player: William Saliba - Points: 164 - Cost: £5.9 million - Team: Arsenal
Player: Benjamin White - Points: 182 - Cost: £6.1 million - Team: Arsenal
Player: Jarrad Branthwaite - Points: 124 - Cost: £4.5 million - Team: Everton
Player: Sergio Gómez - Points: 9 - Cost: £3.8 million - Team: Man City
Player: Jamaal Lascelles - Points: 46 - Cost: £3.7 million - Team: Newcastle
Position: Midfielder
Player: Bukayo Saka - Points: 226 - Cost: £8.8 million - Team: Arsenal
Player: Cole Palmer - Points: 244 - Cost: £6.3 million - Team: Chelsea
Player: Phil Foden - Points: 230 - Cost: £8.5 million - Team: Man City
Player: Anthony Gordon - Points: 183 - Cost: £6.4 million - Team: Newcastle
Player: Son Heung-min - Points: 213 - Cost: £10.0 million - Team: Spurs
Position: Forward (Striker)
Player: Ollie Watkins -